# Generative Adversarial Network (GAN) for MNIST Image Generation

This notebook demonstrates the implementation of a Generative Adversarial Network (GAN) to generate handwritten digit images similar to the MNIST dataset.

## Overview
1. **Import Libraries**: Load necessary PyTorch modules and utilities
2. **Define Generator**: Create a neural network that generates fake images from random noise
3. **Define Discriminator**: Create a neural network that distinguishes real from fake images
4. **Data Preparation**: Load and preprocess MNIST dataset (resized to 14x14 for efficiency)
5. **Training Loop**: Train both networks adversarially 
6. **Results Visualization**: Compare generated images with original MNIST images

## GAN Architecture
- **Generator**: Takes random noise (100 dimensions) → generates 14x14 images
- **Discriminator**: Takes 14x14 images → classifies as real (1) or fake (0)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

## 1. Import Required Libraries

Import essential libraries for GAN implementation:
- **PyTorch**: Core deep learning framework
- **torchvision**: For MNIST dataset loading and image transformations
- **matplotlib**: For visualizing generated vs real images

In [ ]:
# Define the Generator with fewer layers and smaller output size (14x14)
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, 256),
            nn.ReLU(True),
            nn.Linear(256, output_dim),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

## 2. Define Generator Network

The Generator creates fake images from random noise:

**Architecture:**
- **Input**: 100-dimensional random noise vector (latent space)
- **Hidden Layer 1**: 128 neurons with ReLU activation
- **Hidden Layer 2**: 256 neurons with ReLU activation  
- **Output**: 196 neurons (14×14 pixels) with Tanh activation (outputs in [-1,1] range)

The Generator learns to map random noise to realistic-looking digit images.

In [ ]:
# Define the Discriminator with smaller architecture
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

## 3. Define Discriminator Network

The Discriminator classifies images as real or fake:

**Architecture:**
- **Input**: 196 neurons (flattened 14×14 image)
- **Hidden Layer 1**: 256 neurons with LeakyReLU activation (α=0.2)
- **Hidden Layer 2**: 128 neurons with LeakyReLU activation  
- **Output**: 1 neuron with Sigmoid activation (probability: 0=fake, 1=real)

LeakyReLU prevents dying neurons and helps gradient flow during training.

In [ ]:
# Hyperparameters
latent_dim = 100
img_size = 14 * 14  # Reduced image size (14x14)
batch_size = 64
learning_rate = 0.0002
num_epochs = 20  # Fewer epochs for faster training

# Prepare the Data with image resizing to 14x14
transform = transforms.Compose([
    transforms.Resize(14),  # Resize images to 14x14
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

## 4. Set Hyperparameters and Prepare Data

Configure training parameters and load MNIST dataset:

**Hyperparameters:**
- **Latent Dimension**: 100 (size of noise vector input to Generator)
- **Image Size**: 14×14 pixels (reduced from 28×28 for faster training)
- **Batch Size**: 64 images per training batch
- **Learning Rate**: 0.0002 (typical for GAN training)
- **Epochs**: 20 (reduced for demonstration purposes)

**Data Preprocessing:**
- Resize MNIST images from 28×28 to 14×14
- Normalize pixel values to [-1, 1] range (matches Generator's Tanh output)

In [ ]:
# Hyperparameters
latent_dim = 100
img_size = 14 * 14  # Reduced image size (14x14)
batch_size = 64
learning_rate = 0.0002
num_epochs = 20  # Fewer epochs for faster training

# Prepare the Data with image resizing to 14x14
transform = transforms.Compose([
    transforms.Resize(14),  # Resize images to 14x14
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Initialize models, loss function, and optimizers
generator = Generator(input_dim=latent_dim, output_dim=img_size)#.to('cuda')
discriminator = Discriminator(input_dim=img_size)#.to('cuda')

criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate)
optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate)

## 5. Initialize Models and Training Components

Set up the GAN training framework:

**Model Initialization:**
- Create Generator and Discriminator instances
- Both models can be moved to GPU for faster training (commented out for CPU training)

**Training Setup:**
- **Loss Function**: Binary Cross Entropy (BCELoss) - standard for binary classification
- **Optimizers**: Adam optimizers for both networks with learning rate 0.0002
- Adam is preferred over SGD for GAN training due to better convergence properties

In [ ]:
# Training Loop
for epoch in range(num_epochs):
    for i, (imgs, _) in enumerate(train_loader):
        
        # Adversarial ground truths
        valid = torch.ones((imgs.size(0), 1), requires_grad=False)#.to('cuda')
        fake = torch.zeros((imgs.size(0), 1), requires_grad=False)#.to('cuda')
        
        # Configure input
        real_imgs = imgs.view(imgs.size(0), -1)#.to('cuda')
        
        # Train Generator
        optimizer_G.zero_grad()
        z = torch.randn((imgs.size(0), latent_dim))#.to('cuda')
        gen_imgs = generator(z)
        g_loss = criterion(discriminator(gen_imgs), valid)
        g_loss.backward()
        optimizer_G.step()
        
        # Train Discriminator
        optimizer_D.zero_grad()
        real_loss = criterion(discriminator(real_imgs), valid)
        fake_loss = criterion(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

        # Print progress
        if i % 100 == 0:
            print(f"Epoch [{epoch}/{num_epochs}] Batch {i}/{len(train_loader)} \
                  Loss D: {d_loss.item()}, loss G: {g_loss.item()}")

## 6. GAN Training Loop

Implement adversarial training where Generator and Discriminator compete:

**Training Process (for each batch):**

1. **Train Generator:**
   - Generate fake images from random noise
   - Try to fool Discriminator (maximize D(G(z)))
   - Loss: BCE between Discriminator's output on fake images and "real" labels

2. **Train Discriminator:**
   - Train on real images (should output 1)
   - Train on fake images (should output 0)  
   - Loss: Average of real_loss and fake_loss

**Key Points:**
- Use `.detach()` when training Discriminator on fake images to prevent Generator updates
- Monitor both Generator and Discriminator losses to ensure balanced training

In [ ]:
# Visualizing Generated and Original Images
def show_images(gen_images, real_images):
    gen_images = gen_images.view(gen_images.size(0), 1, 14, 14).cpu().data
    real_images = real_images.view(real_images.size(0), 1, 14, 14).cpu().data
    
    # Concatenate generated and real images
    images = torch.cat([gen_images, real_images])
    
    grid = torchvision.utils.make_grid(images, nrow=8, normalize=True)
    
    # Set smaller figure size
    plt.figure(figsize=(6, 6))  # Smaller figure
    plt.imshow(grid.permute(1, 2, 0))
    plt.title('Top Half: Generated Images, Bottom Half: Original Images')
    plt.axis('off')
    plt.show()

# Generate images
z = torch.randn(16, latent_dim)#.to('cuda')  # Reduced to 16 for a smaller plot
gen_imgs = generator(z)

# Get a batch of real images
real_imgs, _ = next(iter(train_loader))
real_imgs = real_imgs[:16].view(16, -1)#.to('cuda')  # Reduced to 16 for a smaller plot

# Show generated vs original images
show_images(gen_imgs, real_imgs)

## 7. Visualize Generated vs Original Images

Compare the quality of generated images with real MNIST digits:

**Visualization Process:**
1. **Generate New Images**: Create 16 fake images from random noise using trained Generator
2. **Get Real Images**: Extract 16 real images from the MNIST dataset
3. **Create Comparison Grid**: 
   - Top half: Generated (fake) images
   - Bottom half: Original (real) images
4. **Display Results**: Show side-by-side comparison to evaluate Generator performance

This visualization helps assess how well the GAN has learned to generate realistic handwritten digits.